In [ ]:
# Datset link
# https://archive.ics.uci.edu/dataset/53/iris

# We will use this dataset from sklearn
# https://scikit-learn.org/dev/modules/generated/sklearn.datasets.load_iris.html#sklearn.datasets.load_iris

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [3]:
iris = load_iris()
df = pd.DataFrame(iris.data, columns=iris.feature_names)
df['target'] = iris.target

In [4]:
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [5]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['target'])

In [9]:
X_train, y_train = train_df.drop(columns=['target']), train_df['target']
X_test, y_test = test_df.drop(columns=['target']), test_df['target']

In [10]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [12]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

In [31]:
class IrisClassifier(nn.Module):
  def __init__(self, input_dim, hidden_dim,output_dim):
    super(IrisClassifier, self).__init__()
    self.network = nn.Sequential(
        nn.Linear(input_dim,hidden_dim),
        nn.ReLU(),
        nn.Linear(hidden_dim,hidden_dim),
        nn.ReLU(),
        nn.Linear(hidden_dim,output_dim),
    )

  def forward(self, x):
    return self.network(x)


In [32]:
input_dim = X_train.shape[1]
hidden_dim = 16
output_dim = 3

In [33]:
model = IrisClassifier(input_dim, hidden_dim, output_dim)

In [34]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [35]:
# Train the model

epochs = 500
for epoch in range(epochs):
  model.train()
  optimizer.zero_grad()

  predictions = model(X_train_tensor)
  loss = criterion(predictions, y_train_tensor)

  loss.backward()
  optimizer.step()

  if (epoch+1) % 10 == 0:
    print(f"Epoch [{epoch+1}/{epochs}], loss : {loss.item():.4f}")

Epoch [10/500], loss : 0.5937
Epoch [20/500], loss : 0.3676
Epoch [30/500], loss : 0.2507
Epoch [40/500], loss : 0.1452
Epoch [50/500], loss : 0.0698
Epoch [60/500], loss : 0.0457
Epoch [70/500], loss : 0.0363
Epoch [80/500], loss : 0.0331
Epoch [90/500], loss : 0.0314
Epoch [100/500], loss : 0.0297
Epoch [110/500], loss : 0.0282
Epoch [120/500], loss : 0.0270
Epoch [130/500], loss : 0.0259
Epoch [140/500], loss : 0.0250
Epoch [150/500], loss : 0.0243
Epoch [160/500], loss : 0.0237
Epoch [170/500], loss : 0.0231
Epoch [180/500], loss : 0.0216
Epoch [190/500], loss : 0.0194
Epoch [200/500], loss : 0.0177
Epoch [210/500], loss : 0.0162
Epoch [220/500], loss : 0.0141
Epoch [230/500], loss : 0.0122
Epoch [240/500], loss : 0.0100
Epoch [250/500], loss : 0.0081
Epoch [260/500], loss : 0.0064
Epoch [270/500], loss : 0.0050
Epoch [280/500], loss : 0.0039
Epoch [290/500], loss : 0.0031
Epoch [300/500], loss : 0.0025
Epoch [310/500], loss : 0.0020
Epoch [320/500], loss : 0.0017
Epoch [330/500], 

In [36]:
model.eval()
with torch.no_grad():
  y_pred = model(X_test_tensor)
  y_pred_labels = torch.argmax(y_pred, dim=1)

  accuracy = (y_pred_labels == y_test_tensor).sum().item() / y_test_tensor.size(0)
  print(f"Test accuracy : {accuracy:.4f}")

Test accuracy : 0.9667


In [37]:
iris.target_names

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

In [38]:
def predict_iris(sepal_length, sepal_width, petal_length, petal_width):
  input_data = np.array([[sepal_length, sepal_width, petal_length, petal_width]])
  input_data = scaler.transform(input_data)
  input_tesnor = torch.tensor(input_data, dtype = torch.float32)

  model.eval()
  with torch.no_grad():
    prediction = model(input_tesnor)
    predicted_class = torch.argmax(prediction, dim=1).item()
  return iris.target_names[predicted_class]

In [39]:
predicted_class = predict_iris(5.1, 3.5, 1.4, 0.2)
print(f"Predicted species : {predicted_class}")

Predicted species : setosa


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
